In [1]:
!pip install -q sentence-transformers

import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics import f1_score
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00:00:0100:01


2025-07-10 15:01:10.478531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752159670.699293      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752159670.765055      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load data
data_path = '/kaggle/input/train-data/final_skill_training_dataset.jsonl 2'

texts = []
labels = []
NUM_SKILLS = 6

with open(data_path, 'r') as f:
    for line in f:
        entry = json.loads(line)
        texts.append(entry["text"])
        label_vector = [0] * NUM_SKILLS
        for l in entry["labels"]:
            label_vector[l] = 1
        labels.append(label_vector)


In [3]:
class SkillDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.tensor(labels).float()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

dataset = SkillDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [4]:
base_model = SentenceTransformer('all-MiniLM-L6-v2')

class SkillClassifier(nn.Module):
    def __init__(self, sentence_model, num_labels):
        super().__init__()
        self.sentence_model = sentence_model
        self.classifier = nn.Linear(sentence_model.get_sentence_embedding_dimension(), num_labels)

    def forward(self, input_texts):
        embeddings = self.sentence_model.encode(input_texts, convert_to_tensor=True)
        logits = self.classifier(embeddings)
        return logits

model = SkillClassifier(base_model, num_labels=NUM_SKILLS)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()
EPOCHS = 3

model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for texts, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 3 Loss: 19.4193


In [10]:
import pandas as pd
import torch
from torch.nn.functional import sigmoid
from tqdm import tqdm
skill_list=['Problem Solving, Critical Thinking, and Analytical Reasoning','Adaptability, Continous Learning, and Creativity', 'Time Management', 'Communication, Teamwork, Collaboration, and Community Engagement', 'Emotional Intelligence and Ethical Decision Making', 'Tech Aptitude']
def predict_skills_from_csv(model, input_csv_path, output_csv_path, threshold=0.5):
    model.eval()
    df = pd.read_csv(input_csv_path)

    results = {
        "Student": [],
        "combined_responses": [],
        "Predicted_Skills": [],
         "Skills":[],
        "Raw_Probabilities": []
    }

    with torch.no_grad():
        for _, row in tqdm(df.iterrows(), total=len(df)):
            text = row['combined_responses']
            student = row['Student']

            logits = model([text])
            probs = sigmoid(logits).squeeze().cpu().numpy()

            predicted_skills = [i for i, p in enumerate(probs) if p > threshold]
            skills=[]
            for i in predicted_skills:
                skills.append(skill_list[i])
            results["Student"].append(student)
            results["combined_responses"].append(text)
            results["Predicted_Skills"].append(predicted_skills)
            results["Skills"].append(skills)
            results["Raw_Probabilities"].append(probs.tolist())

    output_df = pd.DataFrame(results)
    output_df.to_csv(output_csv_path, index=False)
    print("Done")

predict_skills_from_csv(
    model=model,
    input_csv_path="/kaggle/input/onboarding-responses/onboarding_responses.csv",
    output_csv_path="student_skill_predictions.csv"
)


  0%|          | 0/25 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  4%|▍         | 1/25 [00:00<00:02,  8.67it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  8%|▊         | 2/25 [00:00<00:02,  8.87it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 96%|█████████▌| 24/25 [00:03<00:00,  6.67it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:03<00:00,  7.22it/s]

Done
